In [ ]:
import json
from pathlib import Path
from torch.utils.data import DataLoader
import time
import pandas as pd

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer,BertTokenizerFast, BertForQuestionAnswering, DistilBertForQuestionAnswering

In [ ]:
with open('/content/drive/MyDrive/Spring22/CS769/Project/squad/dev-v2.0.json' , 'r') as fp:
  data = json.load(fp)

In [ ]:
data = data['data'] # List of passages 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Spring22/CS769/Project/FInal/DistilBERT/distillbert-test-squad-trained')

model = DistilBertForQuestionAnswering.from_pretrained('/content/drive/MyDrive/Spring22/CS769/Project/FInal/DistilBERT/distillbert-test-squad-trained')
model.eval()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Spring22/CS769/Project/FInal/BERT/bert-test-squad-trained')

model = BertForQuestionAnswering.from_pretrained('/content/drive/MyDrive/Spring22/CS769/Project/FInal/BERT/bert-test-squad-trained')
model.eval()

In [ ]:
def predict(context,query):

  inputs = tokenizer.encode_plus(query, context, return_tensors='pt')

  outputs = model(**inputs)
  # print(outputs)
  answer_start = torch.argmax(outputs[0])  
  # print(answer_start)
  answer_end = torch.argmax(outputs[1]) + 1 
  # print(answer_end)

  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

  return answer

def normalize_text(s):
  """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
  import string, re

  def remove_articles(text):
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)

  def white_space_fix(text):
    return " ".join(text.split())

  def remove_punc(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)

  def lower(text):
    return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
  pred_tokens = normalize_text(prediction).split()
  truth_tokens = normalize_text(truth).split()
  
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)
  
  common_tokens = set(pred_tokens) & set(truth_tokens)
  
  if len(common_tokens) == 0:
    return 0
  
  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)
  
  return 2 * (prec * rec) / (prec + rec)

In [ ]:
def give_an_answer(context,query,answer):

  prediction = predict(context,query)
  em_score = compute_exact_match(prediction, answer)
  f1_score = compute_f1(prediction, answer)

  print(f"Question: {query}")
  print(f"Prediction: {prediction}")
  print(f"True Answer: {answer}")
  print(f"EM: {em_score}")
  print(f"F1: {f1_score}")
  print("\n")
  return f1_score , em_score

In [ ]:
# EXAMPLE

context = data[0]['paragraphs'][0]['context']
qas= data[0]['paragraphs'][0]['qas']

In [ ]:
count = 0
net_f1 = 0
net_em = 0
for i in range(len(qas)):
  if len(qas[i]['answers']) == 0:
    count += 1
    f1_score , em_score = give_an_answer(context, qas[i]['question'] , '')
    net_f1 += f1_score
    net_em += em_score
  else:
    for j in range(len(qas[i]['answers'])):
      count += 1
      f1_score , em_score = give_an_answer(context, qas[i]['question'] , qas[i]['answers'][j]['text'])
      net_f1 += f1_score
      net_em += em_score

print('Total F1 score is ' , net_f1/count)
print('Net EM score is ' , net_em/count)

In [ ]:
context = """ Mount Olympus is the highest mountain in Greece. It is part of the Olympus massif near 
              the Gulf of Thérmai of the Aegean Sea, located in the Olympus Range on the border between 
              Thessaly and Macedonia, between the regional units of Pieria and Larissa, about 80 km (50 mi) 
              southwest from Thessaloniki. Mount Olympus has 52 peaks and deep gorges. The highest peak, 
              Mytikas, meaning "nose", rises to 2917 metres (9,570 ft). It is one of the 
              highest peaks in Europe in terms of topographic prominence. """

queries = [
           "How many metres is Olympus?",
           "Where Olympus is near?",
           "Why did the chicken cross the road?"
          ]
answers = [
           "2917",
           "Gulf of Thérmai of the Aegean Sea",
           "80 km (50 mi)"
          ]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)